In [ ]:
#object_names

label_file = 'object_names.xlsx'

import pandas as pd
import numpy as np
 
# read by default 1st sheet of an excel file
dataframe1 = pd.read_excel(label_file)
labels = np.array(dataframe1).squeeze()
 
print(len(labels))

#for label_pair in labels:
#    print(label_pair[0],label_pair[1])

In [ ]:
path_images = "stimuli_exp/"

from os import listdir
from os.path import isfile, join
filenames = sorted([f for f in listdir(path_images) if isfile(join(path_images, f))])[1:]
len(filenames)

In [ ]:
# OpenAI models:

%env OPENAI_API_KEY= # your key

In [ ]:
import os
from openai import OpenAI
client = OpenAI()

In [ ]:
# GPT-4o

import base64
import requests

from PIL import Image

import time




# OpenAI API Key
api_key = "your key"

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')



headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

previous_label = "nothing"
label_before_that = "nothing"
i = 0
while i < len(labels):
    
    label1 = labels[i][0]

        
    if label1 != previous_label:
        condition = 0
    elif label1 != label_before_that:
        condition = 1
    else:
        condition = 2
          
    
    
    label2 = labels[i][1]
    image_path =filenames[i]
    # Path to your image
    full_image_path = path_images + image_path

    # Getting the base64 string
    base64_image = encode_image(full_image_path)

    prompt = f"""
    What's in this image?
    A. {label1}
    B. {label2}

    Choose either A or B and answer in one or two words.
    """
    
    payload = {
        "model": "gpt-4o",
        "messages": [
          {
            "role": "user",
            "content": [
              {
                "type": "text",
                "text": prompt
              },
              {
                "type": "image_url",
                "image_url": {
                  "url": f"data:image/jpeg;base64,{base64_image}"
                }
              }
            ]
          }
        ],
        "max_tokens": 300,
        "temperature": 0  # temperature controls the randomness of the output
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    
    if 'error' in response.json():
        print("error. pausing for 60sec")
        print("")
        time.sleep(60)
        
    else:
        print()
        print("Id: " + str(i) + ". Condition: " + str(condition)) 
        print(prompt)
        image = Image.open(full_image_path)
        display(image.resize((100,100)))      
        print()
        print("GPT-4o: " + response.json()['choices'][0]['message']['content'])
        label_before_that = previous_label
        previous_label = label1
        i = i+1

In [ ]:
# GPT-4-vision-preview

import base64
import requests

from PIL import Image

import time




# OpenAI API Key
api_key = "your key"

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')



headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

previous_label = "nothing"
label_before_that = "nothing"
i = 0
while i < len(labels):
    
    label1 = labels[i][0]

        
    if label1 != previous_label:
        condition = 0
    elif label1 != label_before_that:
        condition = 1
    else:
        condition = 2
        

    
    
    
    label2 = labels[i][1]
    image_path =filenames[i]
    # Path to your image
    full_image_path = path_images + image_path

    # Getting the base64 string
    base64_image = encode_image(full_image_path)

    prompt = f"""
    What's in this image?
    A. {label1}
    B. {label2}

    Choose either A or B and answer in one or two words.
    """
    
    payload = {
        "model": "gpt-4-vision-preview",
        "messages": [
          {
            "role": "user",
            "content": [
              {
                "type": "text",
                "text": prompt
              },
              {
                "type": "image_url",
                "image_url": {
                  "url": f"data:image/jpeg;base64,{base64_image}"
                }
              }
            ]
          }
        ],
        "max_tokens": 300,
        "temperature": 0  # temperature controls the randomness of the output
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    
    if 'error' in response.json():
        print("error. pausing for 60sec")
        print("")
        time.sleep(60)
        
    else:
        print()
        print("Id: " + str(i) + ". Condition: " + str(condition)) 
        print(prompt)
        image = Image.open(full_image_path)
        display(image.resize((100,100)))      
        print()
        print("GPT-4-vision-preview: " + response.json()['choices'][0]['message']['content'])
        label_before_that = previous_label
        previous_label = label1
        i = i+1

In [ ]:
# Gemini models

GOOGLE_API_KEY = # your key

In [ ]:
pip install -q -U google-generativeai

In [ ]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
# Gemini Flash 1.5

from PIL import Image
    
previous_label = "nothing"
label_before_that = "nothing"    

for i in range(len(labels)):
    label1 = labels[i][0]
    
    if label1 != previous_label:
        condition = 0
    elif label1 != label_before_that:
        condition = 1
    else:
        condition = 2
        
    label2 = labels[i][1]
    image_path = filenames[i]
    full_image_path = path_images + image_path

    prompt = f"""
    What's in this image?
    A. {label1}
    B. {label2}

    Choose either A or B and answer in one or two words.
    """
    
    model = genai.GenerativeModel('gemini-1.5-flash', 
                                 generation_config=genai.GenerationConfig(
        temperature=0,
    ))
    image = Image.open(full_image_path)
    
    try: 
        response = model.generate_content([prompt, image])
        print()
        print("Id: " + str(i) + ". Condition: " + str(condition)) 
        print(prompt)
        display(image.resize((100, 100)))
        print()
        print('Gemini 1.5 Flash: {}'.format(response.text))
        
    except Exception as e:  
        print(f"Error encountered for image {i}: {e}")
  
    label_before_that = previous_label
    previous_label = label1

In [ ]:
# Gemini 1.5 Pro

from PIL import Image
    
previous_label = "nothing"
label_before_that = "nothing"    

for i in range(len(labels)):
    label1 = labels[i][0]
    
    if label1 != previous_label:
        condition = 0
    elif label1 != label_before_that:
        condition = 1
    else:
        condition = 2
        
    label2 = labels[i][1]
    image_path = filenames[i]
    full_image_path = path_images + image_path

    prompt = f"""
    What's in this image?
    A. {label1}
    B. {label2}

    Choose either A or B and answer in one or two words.
    """
    
    model = genai.GenerativeModel('gemini-1.5-pro', 
                                 generation_config=genai.GenerationConfig(
        temperature=0,
    ))
    image = Image.open(full_image_path)
    
    try: 
        response = model.generate_content([prompt, image])
        print()
        print("Id: " + str(i) + ". Condition: " + str(condition)) 
        print(prompt)
        display(image.resize((100, 100)))
        print()
        print('Gemini 1.5 Pro: {}'.format(response.text))
        
    except Exception as e:  
        print(f"Error encountered for image {i}: {e}")
  
    label_before_that = previous_label
    previous_label = label1

In [ ]:
# Claude models

claude_key = # your key


In [ ]:
pip install anthropic

In [ ]:
# claude 3 opus

import anthropic
import base64
from PIL import Image


client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=claude_key,
)

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')
    
    
previous_label = "nothing"
label_before_that = "nothing"
i = 0
while i < len(labels):
    
    label1 = labels[i][0]

        
    if label1 != previous_label:
        condition = 0
    elif label1 != label_before_that:
        condition = 1
    else:
        condition = 2
        
    
    label2 = labels[i][1]
    image_path =filenames[i]
    # Path to your image
    full_image_path = path_images + image_path

    # Getting the base64 string
    base64_image = encode_image(full_image_path)
    image_media_type = "image/png"
    

    prompt = f"""
    What's in this image?
    A. {label1}
    B. {label2}

    Choose either A or B and answer in one or two words.
    """
    
    message = client.messages.create(
        model="claude-3-opus-20240229",
        max_tokens=1000,
        temperature=0,
        messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": image_media_type,
                        "data": base64_image,
                    },
                },
                {
                    "type": "text",
                    "text": prompt
                }
            ],
        }
    ],
)
    
    print()
    print("Id: " + str(i) + ". Condition: " + str(condition)) 
    print(prompt)
    image = Image.open(full_image_path)
    display(image.resize((100,100)))
    print()
    print(f"Claude 3 Opus: {message.content[0].text}")
  
    label_before_that = previous_label
    previous_label = label1
    i = i+1

In [ ]:
# claude 3.5 sonnet

import anthropic
import base64
from PIL import Image


client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=claude_key,
)

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')
    
    
previous_label = "nothing"
label_before_that = "nothing"
i = 0
while i < len(labels):
    
    label1 = labels[i][0]

        
    if label1 != previous_label:
        condition = 0
    elif label1 != label_before_that:
        condition = 1
    else:
        condition = 2
        
    
    label2 = labels[i][1]
    image_path =filenames[i]
    # Path to your image
    full_image_path = path_images + image_path

    # Getting the base64 string
    base64_image = encode_image(full_image_path)
    image_media_type = "image/png"
    

    prompt = f"""
    What's in this image?
    A. {label1}
    B. {label2}

    Choose either A or B and answer in one or two words.
    """
    
    message = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens=1000,
        temperature=0,
        messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": image_media_type,
                        "data": base64_image,
                    },
                },
                {
                    "type": "text",
                    "text": prompt
                }
            ],
        }
    ],
)
    
    print()
    print("Id: " + str(i) + ". Condition: " + str(condition)) 
    print(prompt)
    image = Image.open(full_image_path)
    display(image.resize((100,100)))
    print()
    print(f"Claude 3.5 Sonnet: {message.content[0].text}")
  
    label_before_that = previous_label
    previous_label = label1
    i = i+1